Initial set of functions, which generate the embeddings, models and vectors as required

In [61]:
# Replace with your actual project ID and desired location
project_id = "qwiklabs-gcp-02-c706fd6470f9"
location = "us"
connection_name = "embedding_conn"

# Build and run the command
!bq mk --connection \
  --project_id={project_id} \
  --location={location} \
  --connection_type=CLOUD_RESOURCE \
  {connection_name}


BigQuery error in mk operation: Already Exists: Connection
projects/791998320083/locations/us/connections/embedding_conn


Basic Functions

In [62]:
from google import genai
from google.genai import types
import base64


def generate_contents(chat_history):
    """
    Converts a structured chat history and system instruction into Gemini-compatible contents.

    Args:
        chat_history (list): A list of dicts with keys 'role' and 'content'

    Returns:
        list[types.Content]: Gemini-compatible message sequence.
    """
    contents = [
        types.Content(
            role=entry["role"],
            parts=[types.Part.from_text(text=entry["content"])]
        ) for entry in chat_history
    ]

    return contents


def generate(content, instructions,temp=0):
  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-02-c706fd6470f9",
      location="us-central1",
  )



  model = "gemini-2.0-flash-001" #Use flash because its fast
  contents = content

  generate_content_config = types.GenerateContentConfig(
    temperature = temp, #0 Temp is better for classification
    top_p = 1,
    seed = 0,
    max_output_tokens = 8192,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )],
    system_instruction=[types.Part.from_text(text=instructions)],
  )
  response_text=""
  for chunk in client.models.generate_content_stream(
    model = model,
    contents = contents,
    config = generate_content_config,
    ):
    # print(chunk.text, end="") #No need to print
    response_text += chunk.text
  return response_text.strip()

Embedding and Vector Generation

In [63]:
from google.cloud import bigquery

bq_client = bigquery.Client()


# ----------- 1. Create embedding model ------------
def create_embedding_model(model_path="qwiklabs-gcp-02-c706fd6470f9.jamesw_test.embeddings_model",
                           connection_id="us.embedding_conn",
                           endpoint="text-embedding-005"):
    """
    Creates a remote embedding model in BigQuery using a specified Vertex AI endpoint.
    """
    query = f"""
    CREATE OR REPLACE MODEL `{model_path}`
    REMOTE WITH CONNECTION `{connection_id}`
    OPTIONS (
      ENDPOINT = '{endpoint}'
    );
    """
    bq_client.query(query).result()
    print(f"✅ Created embedding model: {model_path}")


# ----------- 2. Create embedding table ------------
def create_qa_embedding_table(source_table, destination_table, model="qwiklabs-gcp-02-c706fd6470f9.jamesw_test.embeddings_model"):
    """
    Creates a new BigQuery table with embeddings of q + a using the given model.
    """
    query = f"""
  CREATE OR REPLACE TABLE `{destination_table}` AS
  SELECT
    *
  FROM ML.GENERATE_EMBEDDING(
    MODEL `{model}`,
     (
      SELECT
        CONCAT(question, ' ', answer) AS content
      FROM `{source_table}`
    )
  )
  """
    bq_client.query(query).result()
    print(f"✅ Created embedding table: {destination_table}")

# ----------- 3. Embed a query and run similarity search ------------
def run_vector_search(destination_table, query_text, model="qwiklabs-gcp-02-c706fd6470f9.jamesw_test.embeddings_model", top_k=5):
    """
    Embeds a query and finds top K similar rows from the embedding table using cosine similarity.
    """
    query = f"""
    CREATE TEMP TABLE input AS
    SELECT
      ML.GENERATE_EMBEDDING(
        MODEL `{model}`,
        "{query_text}"
      ) AS query_emb;

    SELECT
      q, a,
      (SELECT
          SUM(x * y) / (SQRT(SUM(x * x)) * SQRT(SUM(y * y)))
       FROM UNNEST(t.embedding) AS x WITH OFFSET i
       JOIN UNNEST(input.query_emb) AS y WITH OFFSET j
       ON i = j) AS similarity
    FROM `{destination_table}` t, input
    ORDER BY similarity DESC
    LIMIT {top_k}
    """
    results = bq_client.query(query).to_dataframe()
    return results


One time steps, generate the embedding model, the embeddings, and the vector index.

In [65]:
#Variables
source_table="qwiklabs-gcp-02-c706fd6470f9.jamesw_test.aurora"
destination_table="qwiklabs-gcp-02-c706fd6470f9.jamesw_test.aurora_embedded"
index_name="aurora_vector_index"
embedding_model_path="qwiklabs-gcp-02-c706fd6470f9.jamesw_test.embeddings_model"
# 1. One-time setup
# create_embedding_model()
# create_qa_embedding_table(source_table,destination_table)



In [66]:
# ----------- 4. Main Search Function ------------
def search_embedding_table(table_path, query_text, model_path, top_k=5):
    """
    Embeds the input query and performs a VECTOR_SEARCH against a table of precomputed embeddings.
    """
    query = f"""
    SELECT base.content as content
    FROM VECTOR_SEARCH(
      TABLE `{table_path}`, 'ml_generate_embedding_result',
      (
        SELECT
          ml_generate_embedding_result AS text_embedding,
          "{query_text}" AS query
        FROM ML.GENERATE_EMBEDDING(
          MODEL `{model_path}`,
          (SELECT "{query_text}" AS content)
        )
      ),
      TOP_K => {top_k}
    ) AS result
    """

    return bq_client.query(query).to_dataframe()


Combined Functionality for Chatmode

In [67]:
# ----------- 5. Chat which accounts for RAG and history ------------
def build_chat_prompt_with_history(user_query, retrieved_df, history):
    """
    Builds a multi-turn prompt using retrieved Q&A and prior user-model turns.
    """
    context = "\n".join([f"QnA: {row.content}" for _, row in retrieved_df.iterrows()])

    chat = "\n".join([
        f"User: {entry['content']}" if entry['role'] == "user" else f"Assistant: {entry['content']}"
        for entry in history
    ])

    prompt = f"""

Below are retrieved Q&A pairs to help answer the user's question:

{context}

Here is the conversation so far:
{chat}

Now respond to the user's latest question:
User: {user_query}
"""
    return prompt


# ----------- 6. Generate a chat response ------------
def generate_chat_response(prompt_text):
    contents = generate_contents([
        {"role": "user", "content": prompt_text}
    ])

    instructions = """You are a helpful FAQ bot for the town of Aurora Bay, Alaska. Using the user question and the FAQ context answer the question to the best of your abilities ."""

    return generate(contents, instructions)

# ----------- Everything combined ------------
def multi_turn_chat_step(user_input, chat_history, table_path, index_name, model_path):
    """
    Handles a single multi-turn chat step:
    1. Vector search for relevant context
    2. Build prompt with history
    3. Generate model response
    4. Append to history
    """
    # Retrieve relevant Q&A
    retrieved = search_embedding_table(table_path, user_input, model_path)

    # Build full prompt with retrieved Q&A + chat history
    full_prompt = build_chat_prompt_with_history(user_input, retrieved, chat_history)

    # Run the model
    response = generate_chat_response(full_prompt)

    # Append both turns
    chat_history.append({"role": "user", "content": user_input})
    chat_history.append({"role": "model", "content": response})

    return response






Short Set of Examples

In [68]:
chat_history = []

while True:
    user_msg = input("\nYou: ")
    if user_msg.lower() in ["exit", "quit"]:
        break

    response = multi_turn_chat_step(
        user_input=user_msg,
        chat_history=chat_history,
        table_path=destination_table,
        index_name=index_name,
        model_path=embedding_model_path
    )

    print("\nAssistant:", response)



You: Test

Assistant: This is a test response. I am ready to answer questions about Aurora Bay, Alaska. For example, I can tell you where the Town Hall is located, how to request a building permit, or where to find official town announcements.

You: Tell me about building permits

Assistant: Building permit applications can be obtained at the Town Hall’s Planning & Development Office or on the official website. A site inspection is required before approval.

You: What about schools?

Assistant: Aurora Bay has one elementary school, one middle school, and a combined high school. They’re managed by the Aurora Bay School District. Aurora Bay schools maintain above-average performance in statewide assessments, and the district places a strong emphasis on STEM and environmental education. There’s no university in Aurora Bay itself. The nearest college campus is located in Kodiak, roughly a 2-hour ferry trip away, depending on weather.

You: exit
